# Classification of fraudulent credit card transactions using PyTorch


In [1]:
import pandas as pd                # for reading csv data
import numpy as np                 # for efficient matrix multiplication
from random import shuffle         # for shuffling our datasets
import matplotlib.pyplot as plt    # for plotting the training process
import torch                       # deep learning framework
from torch.utils.data import random_split

ModuleNotFoundError: No module named 'torch'

## Download and read the dataset
You can download the dataset from https://www.kaggle.com/mlg-ulb/creditcardfraud/downloads/creditcard.csv/3

This dataset constists of 284807 examples of credit card transactions. Each example has 28 features as well as the time (to the nearest hour) which it occured. The first column is the time (to the nearest hour) of the transaction, the remaining features are  The last column is the label of whether the transaction was fraudulent (1) or not (0) - this can be interpreted as the true probability of fraud, which is what a perfect model that we will make should predict.

In [ ]:
data = pd.read_csv('creditcard.csv')     # read the data from a spreadsheet
data = np.array(data)                    # convert to matrix so we can use it in the math

#data = data[:, :]                             # you can slice the matrix to only use certain features or examples
'''
labels = data[:, -1]         # the labels are the last column (0 for legitimate or 1 for fraudulent)
features = data[:, :-1]      # crop off the indices and labels
print(features.shape)        # show the shape of the features
print(labels.shape)          # show the shape of the labels
'''
print(data)                  # show the data
print(data.shape)            # show the shape of the data

# What the hell are these features?

The features of this dataset are not the raw features that we might collect about a card transaction, like how many tries it took to get the PIN right, or how quickly the transaction was made etc.
Instead, they are actually a transformation of those original features, such that the new axes point in the direction in which the features vary the most. The new axes are still orthogonal to one another (the transformation is just a rotation of the axes). Transforming the features in this way is called principal component analysis (PCA). Performing PCA should mean that the first few features represent most of the variation of the data, and hence are the most important. This means that we could probably get decent results using only those first few features. This is a form of 'dimensionality reduction'.

This isn't the main focus of the session, but is the explanation for what the features represent, and their strange names (V1, V2... )

# Making the dataset unbiased

Most credit card transactions aren't fraudulent. As such, there are far fewer examples of fraudulent transactions than there are legitimate ones. If we train our model on all of these examples then it will be able to achieve a 99.83% accuracy by just classifying every example as legitimate! 

In order to counteract the bias of the dataset, we will adjust it to make it contain an even number of examples from each class.

In [ ]:
labels = data[:, -1]                # binary vector which has 1s where there is a fraud and 0s otherwise
n_fraud = np.sum(labels)            # sum of the labels is the number of fraudulent examples

# print the stats
print('Number of fraudulent transactions:', n_fraud)
print('Number of legitimate transactions:', len(data) - n_fraud)
print('Percentage of examples that are fraudulent {:.2f}%:'.format((n_fraud / len(data))*100))

In [ ]:
print(data.shape)

fraud_indices = data[:, -1] == 1              # get a boolean vector which has TRUE at the index of each fraudulent 
print(fraud_indices)                      
fraud_examples = data[fraud_indices]      # index the data with the binary vector to get the fraudulent examples

legit_indices = ~fraud_indices            # the tilde (~) inverses the binary vector
print(legit_indices)
legit_examples = data[legit_indices]      # index the data to get the legit examples
clipped_legit_examples = legit_examples[:len(fraud_examples)]       # clip the legit examples so that we have the same number of examples with each label
print(legit_examples.shape)

data = np.vstack((fraud_examples, clipped_legit_examples))    # vertically stack the fraudulent and legit examples into a dataset where there are an even number of examples from each class
print(data.shape)

## Normalising the dataset


In [ ]:
# CENTER AROUND MEAN
# we dont want to normalise the labels! so separate them from the features 
features = data[:, :-1]
labels = data[:, -1]

features -= features.mean(axis=0)       # subtract the mean of each feature (over all rows axis=0) from each feature

# DIVIDE BY RANGE
max_features = features.max(axis=0)       # find the larget value of each feature
min_features = features.min(axis=0)       # find the smallest value of each feature

ranges = max_features - min_features      # find the range of each feature

features /= ranges      # divide by range
print(features)

print(features.shape)
labels = np.reshape(labels, (features.shape[0], 1))       # turn labels from a m-vector into a mx1 matrix so that we can stack
print(labels.shape)

normalised_data = np.hstack((features, labels))     # horizontally stack labels back onto the normalised features


# Split the dataset into training, validation and test sets

It's no use training our model to just perform well on the data we show it. It needs to be able to perform well on unseen examples.

These unseen examples will come from a part of our dataset that we break off into a 'test set'.

For less obvious reasons, we also need to create a 'validation set'. We will see that there are some design choices of our model (hyperparameters, rather than model parameters) that we shouldn't learn from the training set; and if we adjust these based on the model's performance on the test set, then we are training the hyperparameters on the test set... and then performance on the test set no longer becomes representative of performance on unseen examples! The point of the validation set is to train these hyperparameters.

In [ ]:
train_size = int(0.8 * len(normalised_data))
val_size = int(0.15 * len(normalised_data))
test_size = len(normalised_data) - train_size - val_size
train_data, val_data, test_data = random_split(normalised_data, [train_size, val_size, test_size])

# Setting up hyperparameters

Now we have sorted out the data, we are ready to start building the rest of the model.

Firstly we will set some hyperparameters.

Hyperparameters are different to parameters because it doesn't make sense to learn them during training. Some examples include:
- the depth of our model
- the width of our model
- batch size (how many examples we show the model at once)
- learning rate (how much we change our parameters by on each update)
- epochs to train for (how many times we pass the whole dataset through our model)

In [ ]:
batch_size = 16          # how many examples will we pass through our model at once
lr = 0.001               # how big will the step sizes of our model parameter updates be
momentum = 0.6           # what proportion of the previous parameter update will also contribute to the next
epochs = 1               # how many times will we pass our whole dataset through the model

# Create the data loaders

Now we have the dataset, we will create something to pass us the data in mini-batches and shuffle if for us - a data loader.

We want to pass our data to our model in mini-batches (rather than the whole batch at once) for 2 main reasons:
- passing the whole batch through the model will take longer, and slow each training step
- How badly the model performs is a function of the data we pass through it. If we update our model parameters based on what will improve predictions for the batch as a whole, we may actually end up not optimisig for any of them specifically.

This is an implementation from scratch. In the regression example we will use a pre-built class that PyTorch provides.

In [ ]:
class DataLoader():                           # create a data loader to pass our examples to us in batches
    
    def __init__(self, dataset, batch_size):     # what happens when we create an dataloader instance
        # MAKE BATCHES OF DATA
        self.batches = []                        # initialisee empty list of batches
        i = 0                                    # initial index to count where we are counting batches from
        while i + batch_size < len(dataset):     # before we reach the end of the dataset
            self.batches.append(dataset[i:i+batch_size])        # grab a batch from the data and append it to the list of batches
            i += batch_size                      # increase the index to start at the next batch
        self.batches.append(dataset[i:])         # the last batch may not fit into the 
        shuffle(self.batches)                    # shuffle the batches
        
    def __getitem__(self, idx):           # this function is called when we index the data loader e.g. dataset[4]
        if idx == 0:  
            shuffle(self.batches)                # shuffle the batches each epoch
        batch = self.batches[idx]                # get a batch of examples from the list of batches
        features = batch[:, :-1]                 # get the features from that batch (all rows and all columns up until the last one)
        features = torch.tensor(features)        # turn the features into a torch tensor
        features = features.float()              # change the data type
        labels = batch[:, -1]                    # get the labels from the batch (all rows, last column)
        labels = torch.tensor(labels)           
        labels = labels.float()
        labels = labels.unsqueeze(1)             # change from vector to matrix (so the labels come out as the same size as our predictions)
        return features, labels                  # return the features and labels

# CREATE A DATA LOADER
train_loader = DataLoader(train_data, batch_size)          # create a data loader from the normalised dataset of a certain batch size
val_loader = DataLoader(val_data, batch_size)
test_loader = DataLoader(test_data, batch_size)


# SHOW AN EXAMPLE OF A BATCH PRODUCED
print(train_loader[1])
x, y = train_loader[1]
print(x.shape)
print(y.shape)


## Creating the model

We are going to create a function to map our inputs to our output (confidence of transaction being false)

We call the function that we will create to perform this mapping a model, because it should model some ideal function that really maps these types of inputs to these outputs.

We build a neural network of adjustable width and depth, that takes in a vector the size of our inputs and outputs a scalar probabiility of an example being fraudulent.

In [ ]:
# DEFINE SIZES FOR EACH OF OUT NEURAL NETWORKS LAYERS
units1 = 16
units2 = 16
units3 = 16

class NN(torch.nn.Module):                                   # create a neural network class
    
    def __init__(self, n_features=30, n_outputs=1):          # what happens when we create a neural network instance
        super().__init__()                                   # initialise the parent class
        # DEFINE LAYERS TO TAKE FEATURES TO A PROBABILITY OF THIS EXAMPLE BEING FRAUDULENT
        self.layers = torch.nn.Sequential(                   # define the layers of the model sequentially
            torch.nn.Linear(n_features, units1),             # linear layers form weighted combinations of their inputs
            torch.nn.ReLU(),                                 # Rectified Linear Unit activation function
            torch.nn.Linear(units1, units2),
            torch.nn.ReLU(),
            torch.nn.Linear(units2, units3),
            torch.nn.ReLU(),
            torch.nn.Linear(units3, units3),
            torch.nn.ReLU(),
            torch.nn.Linear(units3, units3),
            torch.nn.ReLU(),
            torch.nn.Linear(units3, 1),
            torch.nn.Sigmoid()   # sigmoid function squashes our output in the range 0-1, so it can be a probability
        )
    
    def forward(self, x):          # this function is called when we call our model with some input e.g. model(x)
        x = self.layers(x)         # pass the features of the example through the layers of our model
        return x                   # return the transformed output (should tell us whether we predict fraud or not)
    
nn = NN()           # now actually create an instance of a neural network

# Creating the loss function

The loss function is a measure of how badly the model is currently performing. To measure this when our labels are labels are binary (either 0 or 1) we can use the binary cross entropy loss.

When the label is y=1 (fraudulent) the second term is multiplied by 0 (1-y) and only the first term contributes to the loss. $log(\hat{y})$ increases as the prediction ($\hat{y}$) moves away from 1.

When the label is y=0 (legitimate) the first term is multiplied by 0 (y) and only the second term contributes to the loss. $log(1 - \hat{y})$ increases as the prediction ($\hat{y}$) moves away from 0.

# $L = - [ \ y \ log(\hat{y}) + (1-y) \ log(1 - \hat{y}) \ ]$ 

In [ ]:
criterion = torch.nn.BCELoss()           # binary cross entropy loss function. Can be called to return loss between prediction and label

# Creating the optimiser

The optimiser will update the parameters (weights and biases) of the model in a direction that reduces the error.

We will use stochastic gradient descent (SGD), which updates the weights based on the update rule: 
# $w \leftarrow w - \alpha \frac{\partial L}{\partial w}$ 

This means that the weights are moved in the direction that decreases the loss. The step size is proportional to the gradient by the learning rate (alpha)

In [ ]:
optimiser = torch.optim.Adam(nn.parameters(), lr=lr)#, momentum=momentum)    # define how we will update our weights

# Train the model

Here we repeatedly pass mini-batches of examples through our model, then compare our output to the corresponding labels and update the model weights based on the loss.

In [ ]:
def train(model, epochs=10):                 # define a training function
    train_losses = []                        # initialise an empty list of the losses
    
    batch_idx_for_val = 0
    validation_batch_idxs = []
    validation_losses = []
    
    for epoch in range(epochs):              # for however many epochs we specified
        # TRAINING
        print('Training')
        for batch_idx, batch in enumerate(train_loader):     # for each batch in the training data loader
            features, labels = batch                   # unpack the batch
            print(labels)
            prediction = model(features)               # pass an example's features forward through the model
            print(prediction)
            loss = criterion(prediction, labels)       # calculate the loss
            optimiser.zero_grad()                      # zero the gradients (otherwise they will accumulate)
            loss.backward()                            # find rate of change of loss with respect to model params
            optimiser.step()                           # update the weights of our model
            print('Epoch:', epoch, '\tBatch:', batch_idx, '\tLoss:', loss.item())
            train_losses.append(loss.item())           # add this loss to the list of losses
            
            batch_idx_for_val += 1 # counter for plotting validation losses       
             
            if batch_idx == 1000:    # tell your model to stop at some batch_idx if you want
                #pass
                break
        
        # VALIDATING
        print('Validating')
        val_losses = []
        validation_loss_idxs.append(batch_idx_for_val)
        for batch in val_loader:
            features, labels = batch                   # unpack the batch
            prediction = model(features)               # pass an example's features forward through the model
            val_loss = criterion(prediction, labels)
            val_losses.append(val_loss.item())
        avg_val_loss = np.mean(val_losses)
        validation_losses.append(avg_val_loss)
               
    return train_losses, validation_loss_idxs, val_losses    # return the list of losses from the training function
            
train_loss_list, val_idxs, val_loss_list = train(nn)   # call the training function and store the losses

In [ ]:
plt.plot(train_loss_list)     # plot the training losses
plt.plot(val_idxs, val_loss_list)
plt.show()     # show the training curve